# <font color='blue'> DSA - Python Fundamentos - Capítulo 6 </font>

## Stream de Dados do Twitter com MongoDB, Pandas e Scikit Learn

In [1]:
# Importando Tweepy, Datetime e Json
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
import json

In [2]:
consumer_key = ''

In [3]:
consumer_secret = ''

In [4]:
access_token = ''

In [5]:
access_token_secret = ''

In [6]:
# Criando as chaves de autenticação
auth = OAuthHandler(consumer_key, consumer_secret)

In [7]:
auth.set_access_token(access_token, access_token_secret)

In [8]:
# Criando uma classe para capturar os stream de dados do Twitter e armazenar no MongoDB
class MyListener(StreamListener):
    def on_data(self, dados):
        tweet = json.loads(dados)
        print(tweet)
        created_at = tweet["created_at"]
        id_str = tweet["id_str"]
        text = tweet["text"]
        obj = {"created_at": created_at, "id_str": id_str, "text": text}
        tweetind = col.insert_one(obj).inserted_id
        print(obj)
        return True

In [20]:
mylistener = MyListener()

In [21]:
mystream = Stream(auth, listener= mylistener)

In [22]:
from pymongo import MongoClient

In [23]:
client = MongoClient('localhost', 27017)

In [24]:
db = client.twitterdb

In [25]:
# Criando collection tweets
col = db.tweets

In [26]:
# Criando palavras chaves para buscar nos tweets
keywords = ['Palmeiras', 'verdão', 'Verdão', 'palmeiras']

In [27]:
# Iniciando o filtro e gravando os tweets no MongoDB
mystream.filter(track=keywords)

{'id_str': '810424481676197888', 'truncated': False, 'id': 810424481676197888, 'in_reply_to_user_id_str': None, 'created_at': 'Sun Dec 18 10:00:28 +0000 2016', 'in_reply_to_screen_name': None, 'filter_level': 'low', 'geo': None, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'user': {'id_str': '173006210', 'profile_link_color': '0D0E0F', 'id': 173006210, 'friends_count': 156, 'default_profile': False, 'geo_enabled': True, 'default_profile_image': False, 'profile_background_image_url_https': 'https://pbs.twimg.com/profile_background_images/378800000121352406/800a5ccf0fe7ca856e7b78092a64acbf.jpeg', 'created_at': 'Sat Jul 31 04:47:51 +0000 2010', 'statuses_count': 13389, 'notifications': None, 'profile_use_background_image': True, 'profile_image_url': 'http://pbs.twimg.com/profile_images/806508343690981379/wCoIk1Kh_normal.jpg', 'profile_background_image_url': 'http://pbs.twimg.com/profile_background_images/378800000121352406/800a5ccf0fe7ca8

KeyboardInterrupt: 

## Aperte o botão STOP na barra de ferramentas para parar a coleta de dados no Twitter

In [28]:
col.find_one()

{'_id': ObjectId('5854778eaeed02107c6859fa'),
 'created_at': 'Fri Dec 16 23:23:56 +0000 2016',
 'id_str': '809901908001521668',
 'text': 'RT @pythontrending: bowtie - create a dashboard with python! https://t.co/uyWtHldB8V'}

In [29]:
# Criando um dataset retornando os dados do MongoDB
dataset = [{
        'created_at': item['created_at'],
        'text': item['text']
    } for item in col.find()]

In [30]:
import pandas as pd

In [31]:
df = pd.DataFrame(dataset)

In [32]:
df

,created_at,text
0,Fri Dec 16 23:23:56 +0000 2016,RT @pythontrending: bowtie - create a dashboar...
1,Fri Dec 16 23:23:59 +0000 2016,Tweets and python BFF &lt;3 2446280.807651111
2,Fri Dec 16 23:24:00 +0000 2016,Manually raising (throwing) an exception in Py...
3,Fri Dec 16 23:24:00 +0000 2016,RT @VisualStudio: Here's a post and video from...
4,Fri Dec 16 23:24:06 +0000 2016,RT @agramfort: It looks like I forgot to tell ...
5,Fri Dec 16 23:24:11 +0000 2016,RT @dyjh: My new book out in print. Listed #Vo...
6,Fri Dec 16 23:24:15 +0000 2016,It's interesting how recent studies are suppor...
7,Fri Dec 16 23:24:24 +0000 2016,Santé et Big Data : une véritable course d'obs...
8,Fri Dec 16 23:24:31 +0000 2016,@landerso30 there is never enough data when th...
9,Fri Dec 16 23:24:36 +0000 2016,RT @MegaDataMama: 'Big Data' Questioned in Wak...


In [33]:
# Importando o Scikit Learn 
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
# Usando o método CountVectorizer para criar uma matriz de documentos
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.text)

In [36]:
# Contando o número de ocorrências das principais palavras em nosso dataset
word_count = pd.DataFrame(cv.get_feature_names(), columns=['word'])
word_count['count'] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)
word_count[:50]

,word,count
0,co,16
1,https,16
2,rt,11
3,python,10
4,data,7
5,palmeiras,5
6,of,4
7,and,4
8,no,3
9,in,3


# Fim